In [ ]:
#Byte-at-a-time ECB decryption (Harder)
import random
from random import randint
import os
import string
# The below file is given by OPEN SSL
!pip install pycrypto
from Crypto.Cipher import AES
from math import ceil
from binascii import unhexlify, b2a_base64, a2b_base64, hexlify
import chardet
from os.path import commonprefix

     |████████████████████████████████| 450kB 10.6MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499931 sha256=5ab650d4eb0d5bb1c2d6b88d769a7efeac40a8d052922411b5a37297c9b48752
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built pycrypto


Please Note code is as it is copyied from the Byte-at-a-time ECB decryption (Simple) CHALLANGE 12 SET 2 version with some modification as mentioned in the comment

In [ ]:
def pkcs7(message, blocksize):
  padlen = blocksize-len(message)%blocksize
  if(padlen == blocksize):
    padlen = 0
  pkcsmessage=bytes([padlen])*padlen
  return message+pkcsmessage

In [ ]:
def random_byte_create(lenofbytes):
  randomnumber=[]
  for i in range(lenofbytes):
    randomnumber.append(random.randint(0,255))
  return bytes(randomnumber)

In [ ]:
Key_global=random_byte_create(16)
given_plain_text=bytes("Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK",'utf-8')
random_size=160
random_string=random_byte_create(random.randint(1,80))


In [ ]:
def encypt_oracle(message):
  #First convert the given plain text in base 64
  base64_plaintext=a2b_base64(given_plain_text)
  # Given your-random string||string || unknown-string
  new_message=random_string+message + base64_plaintext
  #ECB Encryption
  plaintext=pkcs7(new_message,16)
  cipher_object=AES.new(Key_global,AES.MODE_ECB)
  return cipher_object.encrypt(plaintext)

In [ ]:
def bytes_to_str(message):
    return "".join(filter(lambda x: x in string.printable, "".join(map(chr, message))))

In [ ]:
def guess_block_size():
    length_output = len(encypt_oracle(b'A'*0))
    for i in range(1, random_size):
        new_length_output = len(encypt_oracle(b'A'*i))
        block_size = new_length_output - length_output
        if block_size != 0:
            break
        length_output = new_length_output
    return length_output, block_size

def guess_num_random_blocks(length_output, block_size):
    common_prefix_length = len(commonprefix([encypt_oracle(b''), encypt_oracle(b'A')]))
    for num_random_blocks in range(int(length_output/block_size)):
        if common_prefix_length < block_size * num_random_blocks:
            break
    return num_random_blocks
def guess_string_lengths(block_size, num_random_blocks):
    encrypted_strings =[]
    for i in range(block_size):
        encrypted_strings.append(encypt_oracle(b'A'*i))
        random_string_length = len(commonprefix(encrypted_strings))
        unknown_string_length = len(encrypted_strings[0]) - random_string_length
        if len(encrypted_strings) > 1 and random_string_length >= block_size * num_random_blocks and random_string_length % block_size == 0:
            min_addition = i-1
            break
        encrypted_strings = [encrypted_strings[-1]]
    return min_addition, random_string_length, unknown_string_length

In [ ]:
#Detect the Encryption mechanism of the block
def guess_aes_ecb(ciphertext,Block_size):
  #The logic is simple, In ECB the block content will be same , becuase for same message the cipher text will be same and not randomized
  #This formula only fails when messages are all unique
  blocks_of_ciphertext=[]
  number_of_blocks=len(ciphertext)//Block_size
  for i in range(number_of_blocks):
    blocks_of_ciphertext.append(ciphertext[i:i+Block_size])
  unique_num_blocks=set(blocks_of_ciphertext)
  #Returns True for ECB otherwise false
  return len(unique_num_blocks)<len(ciphertext)

In [ ]:
def guess_first_byte():
  length_output, block_size = guess_block_size()
  num_random_blocks = guess_num_random_blocks(length_output, block_size)
  min_addition, random_string_length, unknown_string_length = guess_string_lengths(block_size, num_random_blocks)
  #This logic contains 3 Section
  # 1 )Discover the unknownblock size of the ciphertext I have taken arbitraty number of iterations, it can be infinity also
   # Check if its ECB or not with the unknown block size
  if not guess_aes_ecb(encypt_oracle(b'A'*random_size*5),block_size):
    return b'The Given Text is Not in ECB Mode'
  # Otherwise follow rest of the steps, find the unknown block length
  new_ciphertext_blocksize=block_size
  final_block_length=unknown_string_length//new_ciphertext_blocksize
  unknown_string = b''
  input_block=b'A'* (new_ciphertext_blocksize+min_addition)
  for block_number in range(final_block_length):
    unknown_string_block = b''
    # First new_ciphertext_blocksize-1 blocks becomes random string in A-->check one_byte_short line , then the last block will have the random UNKNOWN STRING
    # Again we do the above step but now new_ciphertext_blocksize-2 blocks becomes random string--> check one_byte_short, then the last block will have the random UNKNOWN STRING
    # This we repeat again and again 
    for n in range(new_ciphertext_blocksize + min_addition-1, min_addition-1, -1):
        input_block = input_block[1:] 
        last_byte_dict = {encypt_oracle(input_block + bytes([b]))[random_string_length:random_string_length+new_ciphertext_blocksize]: bytes([b]) for b in range(256)}
        one_byte_short = encypt_oracle(b'A'*n)[(block_number*new_ciphertext_blocksize)+random_string_length:(block_number*new_ciphertext_blocksize)+random_string_length+new_ciphertext_blocksize]
        if one_byte_short not in last_byte_dict:
          break
        last_byte = last_byte_dict[one_byte_short]
        unknown_string_block += last_byte
        input_block += last_byte
    unknown_string+=unknown_string_block
    input_block=b'A'*min_addition+unknown_string_block
  return unknown_string



In [ ]:
if __name__== "__main__":
  for line in bytes_to_str(guess_first_byte()).split("\n"):
    print(line)

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by

